In [1]:
import csv
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import sys
import re
import json

In [5]:
news_articles_root = 'https://www.livehindustan.com'
#because access was denied we use header to access it
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

# Settings

In [6]:
cities = ['varanasi','azamgarh','mau','ghazipur','gorakhpur','prayagraj','pratapgarh-kunda','bhadohi'
         'deoria','jaunpur','mirzapur','robertsganj-kotwali','kausambi','raebareli','chandauli','balia','amethi',
          'maharajganj','balrampur','gonda']

In [7]:
pages = 3 #Actually a list would be good for each city

# Scraping

In [ ]:
for city in cities:
    
    times = []
    titles = []
    contents = []
    
    for page in range(0,pages+1):
        
        markup = requests.get(news_articles_root + '/uttar-pradesh/'+city+'/news-'+str(page),headers = agent).text
        soup = BeautifulSoup(markup, "lxml")
        links = soup.find_all('h4',{'class':'hindustan-link'})
        dates = soup.find_all('div',{'class':'list-time-tags tags-list'})

        for i in range(0,len(links)):
            news_link = links[i].find('a',href = True)['href'] 
            news_title = links[i].find('a',href = True)['title'].replace('\xa0',' ')
            news_date = dates[i].find('span').get_text()
            news_markup = requests.get(news_articles_root + news_link,headers = agent).text
            news_soup = BeautifulSoup(news_markup, "lxml")
            news_div_content = news_soup.find('div',{'class':'story-page-content'})
            news_content = news_div_content.get_text().replace('\xa0',' ')

            times.append(news_date)
            titles.append(news_title)
            contents.append(news_content)
    
    news = {'Time':times,'Title':titles,'Content':contents}
    df = pd.DataFrame(news,columns = ['Time','Title','Content'])
    df.to_csv(city +'.csv')